In [1]:
%%capture
!pip install torch-geometric

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from sklearn.metrics import f1_score, accuracy_score
import networkx as nx
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

In [3]:
from torch_geometric.datasets import Planetoid

# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="CiteSeer")

# Print information about the dataset
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {dataset[0].x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Has isolated nodes: {dataset[0].has_isolated_nodes()}')

Processing...


Number of graphs: 1
Number of nodes: 3327
Number of features: 3703
Number of classes: 6
Has isolated nodes: True


Done!


## GCN

In [4]:
test_mask = dataset.test_mask
val_mask = dataset.val_mask
train_mask = ~(test_mask + val_mask)

In [196]:
class ModelCFG:
        model ='GCN'
        num_labels = 6
        scheduler= False
        warnap = False
        max_epoches=175
        lr = 3e-4
        eps=1e-6
        betas=(0.9, 0.999)
        drop = 0.1
        depths = [3703,256]
        act = nn.GELU()
        _act = 'gelu'
        weight_decay = 1e-4
        
cfg = ModelCFG()

In [197]:
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv

class GCNModel(nn.Module):
    def __init__(self,CFG):
        super().__init__()
        self.convs = nn.ModuleList()
        self.act = CFG.act
        self.drop = nn.Dropout(CFG.drop)
        for i in range(len(CFG.depths)-1):
            self.convs.append(GCNConv(CFG.depths[i],CFG.depths[i+1]))
        self.head = nn.Linear(CFG.depths[-1],CFG.num_labels)
    
    def forward(self, x, edge_index):
        x = self.drop(x)
        for conv in self.convs:
            x = conv(x,edge_index)
            x = self.act(x)
        x = self.head(x).softmax(dim=-1)
        return x

In [198]:
def train(model, data, criterion, optimizer, epochs):
    

    model.train()
    for epoch in trange(epochs):
        
        pl.seed_everything(56)
        optimizer.zero_grad()
        logits = model(data.x.cuda(), data.edge_index.cuda())
        loss = criterion(logits[train_mask], data.y[train_mask].cuda())
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            val_acc = accuracy_score(logits[val_mask].argmax(dim=1).cpu().detach().numpy(),
                                     data.y[val_mask].cpu().detach().numpy())
            print(val_acc)
    
    test_acc = accuracy_score(logits[test_mask].argmax(dim=1).cpu().detach().numpy(),
                             data.y[test_mask].cpu().detach().numpy())
    
    print(f'Test Score {test_acc}')
    return val_acc, test_acc

In [199]:
# Experements
if False:
    Experements = {
        'epoches': [],
        'lr': [],
        'drop': [],
        'depths': [],
        'act': [],
        'weight_decay': [],
        'test_score': [],
        'val_score': []
    }

def ExperementsUpgrade(cfg,test_score,val_score,Experements):
    Experements['epoches'].append(cfg.max_epoches)
    Experements['lr'].append(cfg.lr)
    Experements['drop'].append(cfg.drop)
    Experements['depths'].append(cfg.depths)
    Experements['act'].append(cfg._act)
    Experements['weight_decay'].append(cfg.weight_decay)
    Experements['test_score'].append(test_score)
    Experements['val_score'].append(val_score)
    return Experements

In [200]:
model = GCNModel(cfg).cuda()

optimizer = torch.optim.AdamW(model.parameters(),
                              lr = cfg.lr,
                              eps = cfg.eps,
                              betas = cfg.betas,
                              weight_decay = cfg.weight_decay)

criterion = nn.CrossEntropyLoss()

In [201]:
val_acc, test_acc = train(model, dataset, criterion, optimizer, cfg.max_epoches)
Experements = ExperementsUpgrade(cfg,test_acc,val_acc,Experements)

  0%|          | 0/175 [00:00<?, ?it/s]

0.106
0.68
0.726
0.734
0.744
0.748
0.762
0.776
0.778
0.78
0.776
0.784
0.784
0.784
0.782
0.786
0.788
0.782
Test Score 0.789


In [202]:
pd.DataFrame(Experements)

,epoches,lr,drop,depths,act,weight_decay,test_score,val_score
0,200,0.0003,0.1,"[3703, 64]",gelu,0.0001,0.777,0.780
1,175,0.0003,0.1,"[3703, 64]",gelu,0.0001,0.778,0.778
2,150,0.0003,0.1,"[3703, 64]",gelu,0.0001,0.779,0.778
3,100,0.0003,0.1,"[3703, 64]",gelu,0.0001,0.752,0.768
4,250,0.0003,0.1,"[3703, 64]",gelu,0.0001,0.775,0.774
5,200,0.0003,0.2,"[3703, 64]",gelu,0.0001,0.769,0.774
6,250,0.0003,0.2,"[3703, 64]",gelu,0.0001,0.770,0.764
7,150,0.0003,0.2,"[3703, 64]",gelu,0.0001,0.770,0.774
8,200,0.0003,0.0,"[3703, 64]",gelu,0.0001,0.776,0.782
9,175,0.0003,0.0,"[3703, 64]",gelu,0.0001,0.778,0.782


## GAT

In [347]:
test_mask = dataset.test_mask
val_mask = dataset.val_mask
train_mask = ~(test_mask + val_mask)

In [454]:
class ModelCFG:
        model ='GAT'
        num_labels = 6
        scheduler= False
        warnap = False
        max_epoches=200
        lr = 2e-4
        eps=1e-6
        betas=(0.9, 0.999)
        drop = 0.0
        depths = [3703,64,16]
        heads = [8,2]
        act = nn.GELU()
        _act = 'gelu'
        weight_decay = 1e-4
        
cfg = ModelCFG()

In [455]:
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv

class GATModel(nn.Module):
    def __init__(self,CFG):
        super().__init__()
        self.convs = nn.ModuleList()
        self.act = CFG.act
        self.drop = nn.Dropout(CFG.drop)
        for i in range(len(CFG.depths)-1):
            if i == 0:
                self.convs.append(GATConv(CFG.depths[i],CFG.depths[i+1],heads=CFG.heads[i]))
            else:
                self.convs.append(GATConv(CFG.depths[i] * CFG.heads[i-1],CFG.depths[i+1],heads=CFG.heads[i]))
        self.head = nn.Linear(CFG.depths[-1] * CFG.heads[-1],CFG.num_labels)
    
    def forward(self, x, edge_index):
        x = self.drop(x)
        for conv in self.convs:
            x = conv(x,edge_index)
            x = self.act(x)
        x = self.head(x).softmax(dim=-1)
        return x

In [456]:
def train(model, data, criterion, optimizer, epochs):
    

    model.train()
    for epoch in trange(epochs):
        
        pl.seed_everything(56)
        optimizer.zero_grad()
        logits = model(data.x.cuda(), data.edge_index.cuda())
        loss = criterion(logits[train_mask], data.y[train_mask].cuda())
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            val_acc = accuracy_score(logits[val_mask].argmax(dim=1).cpu().detach().numpy(),
                                     data.y[val_mask].cpu().detach().numpy())
            print(val_acc)
    
    test_acc = accuracy_score(logits[test_mask].argmax(dim=1).cpu().detach().numpy(),
                             data.y[test_mask].cpu().detach().numpy())
    
    print(f'Test Score {test_acc}')
    return val_acc, test_acc

In [457]:
# Experements
if False:
    Experements = {
        'epoches': [],
        'lr': [],
        'drop': [],
        'depths': [],
        'act': [],
        'weight_decay': [],
        'test_score': [],
        'val_score': [],
        'heads': []
    }

def ExperementsUpgrade(cfg,test_score,val_score,Experements):
    Experements['epoches'].append(cfg.max_epoches)
    Experements['lr'].append(cfg.lr)
    Experements['drop'].append(cfg.drop)
    Experements['depths'].append(cfg.depths)
    Experements['act'].append(cfg._act)
    Experements['weight_decay'].append(cfg.weight_decay)
    Experements['test_score'].append(test_score)
    Experements['val_score'].append(val_score)
    Experements['heads'].append(cfg.heads)
    return Experements

In [458]:
model = GATModel(cfg).cuda()

optimizer = torch.optim.AdamW(model.parameters(),
                              lr = cfg.lr,
                              eps = cfg.eps,
                              betas = cfg.betas,
                              weight_decay = cfg.weight_decay)

criterion = nn.CrossEntropyLoss()

In [459]:
val_acc, test_acc = train(model, dataset, criterion, optimizer, cfg.max_epoches)
Experements = ExperementsUpgrade(cfg,test_acc,val_acc,Experements)

  0%|          | 0/200 [00:00<?, ?it/s]

0.188
0.288
0.502
0.604
0.616
0.62
0.634
0.676
0.736
0.774
0.78
0.794
0.79
0.792
0.798
0.798
0.786
0.784
0.784
0.784
Test Score 0.765


In [460]:
pd.DataFrame(Experements)

,epoches,lr,drop,depths,act,weight_decay,test_score,val_score,heads
0,175,0.0003,0.0,"[3703, 64]",gelu,0.0001,0.776,0.766,[8]
1,200,0.0003,0.0,"[3703, 64]",gelu,0.0001,0.763,0.758,[8]
2,150,0.0003,0.0,"[3703, 64]",gelu,0.0001,0.779,0.766,[8]
3,125,0.0003,0.0,"[3703, 64]",gelu,0.0001,0.778,0.776,[8]
4,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.784,0.770,[8]
5,150,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.771,0.788,[8]
6,200,0.0002,0.1,"[3703, 64]",gelu,0.0001,0.776,0.774,[8]
7,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.768,0.778,[4]
8,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.775,0.774,[2]
9,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.776,0.772,[6]


## GATV2

In [471]:
test_mask = dataset.test_mask
val_mask = dataset.val_mask
train_mask = ~(test_mask + val_mask)

In [572]:
class ModelCFG:
        model ='GATV2'
        num_labels = 6
        scheduler= False
        warnap = False
        max_epoches=200
        lr = 2e-4
        eps=1e-6
        betas=(0.9, 0.999)
        drop = 0.0
        depths = [3703,32]
        heads = [16]
        act = nn.GELU()
        _act = 'gelu'
        weight_decay = 1e-4
        
cfg = ModelCFG()

In [573]:
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv

class GATV2Model(nn.Module):
    def __init__(self,CFG):
        super().__init__()
        self.convs = nn.ModuleList()
        self.act = CFG.act
        self.drop = nn.Dropout(CFG.drop)
        for i in range(len(CFG.depths)-1):
            if i == 0:
                self.convs.append(GATv2Conv(CFG.depths[i],CFG.depths[i+1],heads=CFG.heads[i]))
            else:
                self.convs.append(GATv2Conv(CFG.depths[i] * CFG.heads[i-1],CFG.depths[i+1],heads=CFG.heads[i]))
        self.head = nn.Linear(CFG.depths[-1] * CFG.heads[-1],CFG.num_labels)
    
    def forward(self, x, edge_index):
        x = self.drop(x)
        for conv in self.convs:
            x = conv(x,edge_index)
            x = self.act(x)
        x = self.head(x).softmax(dim=-1)
        return x

In [574]:
def train(model, data, criterion, optimizer, epochs):
    

    model.train()
    for epoch in trange(epochs):
        
        pl.seed_everything(56)
        optimizer.zero_grad()
        logits = model(data.x.cuda(), data.edge_index.cuda())
        loss = criterion(logits[train_mask], data.y[train_mask].cuda())
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            val_acc = accuracy_score(logits[val_mask].argmax(dim=1).cpu().detach().numpy(),
                                     data.y[val_mask].cpu().detach().numpy())
            print(val_acc)
    
    test_acc = accuracy_score(logits[test_mask].argmax(dim=1).cpu().detach().numpy(),
                             data.y[test_mask].cpu().detach().numpy())
    
    print(f'Test Score {test_acc}')
    return val_acc, test_acc

In [575]:
# Experements
if False:
    Experements = {
        'epoches': [],
        'lr': [],
        'drop': [],
        'depths': [],
        'act': [],
        'weight_decay': [],
        'test_score': [],
        'val_score': [],
        'heads': []
    }

def ExperementsUpgrade(cfg,test_score,val_score,Experements):
    Experements['epoches'].append(cfg.max_epoches)
    Experements['lr'].append(cfg.lr)
    Experements['drop'].append(cfg.drop)
    Experements['depths'].append(cfg.depths)
    Experements['act'].append(cfg._act)
    Experements['weight_decay'].append(cfg.weight_decay)
    Experements['test_score'].append(test_score)
    Experements['val_score'].append(val_score)
    Experements['heads'].append(cfg.heads)
    return Experements

In [576]:
model = GATV2Model(cfg).cuda()

optimizer = torch.optim.AdamW(model.parameters(),
                              lr = cfg.lr,
                              eps = cfg.eps,
                              betas = cfg.betas,
                              weight_decay = cfg.weight_decay)

criterion = nn.CrossEntropyLoss()

In [577]:
val_acc, test_acc = train(model, dataset, criterion, optimizer, cfg.max_epoches)
Experements = ExperementsUpgrade(cfg,test_acc,val_acc,Experements)

  0%|          | 0/200 [00:00<?, ?it/s]

0.274
0.604
0.672
0.69
0.702
0.718
0.728
0.756
0.768
0.776
0.784
0.784
0.792
0.792
0.794
0.792
0.792
0.782
0.778
0.768
Test Score 0.791


In [578]:
pd.DataFrame(Experements)

,epoches,lr,drop,depths,act,weight_decay,test_score,val_score,heads
0,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.784,0.766,[8]
1,175,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.782,0.780,[8]
2,150,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.776,0.790,[8]
3,150,0.0002,0.0,"[3703, 32]",gelu,0.0001,0.774,0.784,[8]
4,200,0.0002,0.0,"[3703, 64, 16]",gelu,0.0001,0.777,0.782,"[8, 4]"
5,200,0.0002,0.0,"[3703, 64, 32]",gelu,0.0001,0.773,0.774,"[8, 4]"
6,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.786,0.766,[9]
7,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.779,0.774,[10]
8,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.792,0.772,[7]
9,200,0.0002,0.0,"[3703, 64]",gelu,0.0001,0.786,0.774,[6]


# Graph Sage

In [4]:
test_mask = dataset.test_mask
val_mask = dataset.val_mask
train_mask = ~(test_mask + val_mask)

In [259]:
class ModelCFG:
        model ='GraphSage'
        num_labels = 6
        scheduler= False
        warnap = False
        max_epoches=150
        aggr = 'mean'
        lr = 3e-4
        eps=1e-6
        betas=(0.9, 0.999)
        drop = 0.0
        depths = [3703,128]
        act = nn.GELU()
        _act = 'gelu'
        weight_decay = 2e-4
        
cfg = ModelCFG()

In [260]:
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv, SAGEConv

class GraphSageModel(nn.Module):
    def __init__(self,CFG):
        super().__init__()
        self.convs = nn.ModuleList()
        self.act = CFG.act
        self.drop = nn.Dropout(CFG.drop)
        for i in range(len(CFG.depths)-1):
            self.convs.append(SAGEConv(CFG.depths[i],CFG.depths[i+1],aggr=CFG.aggr))
        self.head = nn.Linear(CFG.depths[-1],CFG.num_labels)
    
    def forward(self, x, edge_index):
        x = self.drop(x)
        for conv in self.convs:
            x = conv(x,edge_index)
            x = self.act(x)
        x = self.head(x).softmax(dim=-1)
        return x

In [261]:
def train(model, data, criterion, optimizer, epochs):
    

    model.train()
    for epoch in trange(epochs):
        
        pl.seed_everything(56)
        optimizer.zero_grad()
        logits = model(data.x.cuda(), data.edge_index.cuda())
        loss = criterion(logits[train_mask], data.y[train_mask].cuda())
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            val_acc = accuracy_score(logits[val_mask].argmax(dim=1).cpu().detach().numpy(),
                                     data.y[val_mask].cpu().detach().numpy())
            print(val_acc)
    
    test_acc = accuracy_score(logits[test_mask].argmax(dim=1).cpu().detach().numpy(),
                             data.y[test_mask].cpu().detach().numpy())
    
    print(f'Test Score {test_acc}')
    return val_acc, test_acc

In [262]:
# Experements
if False:
    Experements = {
        'epoches': [],
        'lr': [],
        'drop': [],
        'depths': [],
        'act': [],
        'aggr':[],
        'weight_decay': [],
        'test_score': [],
        'val_score': []
    }

def ExperementsUpgrade(cfg,test_score,val_score,Experements):
    Experements['epoches'].append(cfg.max_epoches)
    Experements['lr'].append(cfg.lr)
    Experements['drop'].append(cfg.drop)
    Experements['depths'].append(cfg.depths)
    Experements['act'].append(cfg._act)
    Experements['weight_decay'].append(cfg.weight_decay)
    Experements['test_score'].append(test_score)
    Experements['val_score'].append(val_score)
    Experements['aggr'].append(cfg.aggr)
    return Experements

In [263]:
model = GraphSageModel(cfg).cuda()

optimizer = torch.optim.AdamW(model.parameters(),
                              lr = cfg.lr,
                              eps = cfg.eps,
                              betas = cfg.betas,
                              weight_decay = cfg.weight_decay)

criterion = nn.CrossEntropyLoss()

In [264]:
val_acc, test_acc = train(model, dataset, criterion, optimizer, cfg.max_epoches)
Experements = ExperementsUpgrade(cfg,test_acc,val_acc,Experements)

  0%|          | 0/150 [00:00<?, ?it/s]

0.136
0.632
0.69
0.714
0.722
0.736
0.756
0.772
0.782
0.778
0.788
0.786
0.782
0.78
0.78
Test Score 0.779


In [266]:
pd.DataFrame(Experements)

,epoches,lr,drop,depths,act,aggr,weight_decay,test_score,val_score
0,150,0.0003,0.10,"[3703, 64]",gelu,mean,0.0001,0.770,0.766
1,100,0.0003,0.10,"[3703, 64]",gelu,mean,0.0001,0.762,0.764
2,125,0.0003,0.10,"[3703, 64]",gelu,mean,0.0001,0.766,0.766
3,175,0.0003,0.10,"[3703, 64]",gelu,mean,0.0001,0.768,0.766
4,200,0.0003,0.10,"[3703, 64]",gelu,mean,0.0001,0.762,0.764
5,150,0.0003,0.00,"[3703, 64]",gelu,mean,0.0001,0.779,0.778
6,200,0.0003,0.00,"[3703, 64]",gelu,mean,0.0001,0.770,0.776
7,100,0.0003,0.00,"[3703, 64]",gelu,mean,0.0001,0.763,0.784
8,125,0.0003,0.00,"[3703, 64]",gelu,mean,0.0001,0.775,0.776
9,150,0.0003,0.05,"[3703, 64]",gelu,mean,0.0001,0.766,0.764


# GinConv

In [267]:
test_mask = dataset.test_mask
val_mask = dataset.val_mask
train_mask = ~(test_mask + val_mask)

In [382]:
class ModelCFG:
        model ='GinConv'
        num_labels = 6
        scheduler= False
        warnap = False
        max_epoches=80
        lr = 3e-4
        eps=1e-6
        betas=(0.9, 0.999)
        drop = 0.0
        depths = [3703,80]
        act = nn.GELU()
        _act = 'gelu'
        weight_decay = 2e-4
        
cfg = ModelCFG()

In [383]:
from torch_geometric.nn import GINConv

class GINModel(nn.Module):
    def __init__(self,CFG):
        super().__init__()
        self.convs = nn.ModuleList()
        self.act = CFG.act
        self.drop = nn.Dropout(CFG.drop)
        for i in range(len(CFG.depths)-1):
            self.convs.append(GINConv(nn.Linear(CFG.depths[i],CFG.depths[i+1])))
        self.head = nn.Linear(CFG.depths[-1],CFG.num_labels)
    
    def forward(self, x, edge_index):
        x = self.drop(x)
        for conv in self.convs:
            x = conv(x,edge_index)
            x = self.act(x)
        x = self.head(x).softmax(dim=-1)
        return x

In [384]:
def train(model, data, criterion, optimizer, epochs):
    

    model.train()
    for epoch in trange(epochs):
        
        pl.seed_everything(56)
        optimizer.zero_grad()
        logits = model(data.x.cuda(), data.edge_index.cuda())
        loss = criterion(logits[train_mask], data.y[train_mask].cuda())
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            val_acc = accuracy_score(logits[val_mask].argmax(dim=1).cpu().detach().numpy(),
                                     data.y[val_mask].cpu().detach().numpy())
            print(val_acc)
    
    test_acc = accuracy_score(logits[test_mask].argmax(dim=1).cpu().detach().numpy(),
                             data.y[test_mask].cpu().detach().numpy())
    
    print(f'Test Score {test_acc}')
    return val_acc, test_acc

In [385]:
# Experements
if False:
    Experements = {
        'epoches': [],
        'lr': [],
        'drop': [],
        'depths': [],
        'act': [],
        'weight_decay': [],
        'test_score': [],
        'val_score': []
    }

def ExperementsUpgrade(cfg,test_score,val_score,Experements):
    Experements['epoches'].append(cfg.max_epoches)
    Experements['lr'].append(cfg.lr)
    Experements['drop'].append(cfg.drop)
    Experements['depths'].append(cfg.depths)
    Experements['act'].append(cfg._act)
    Experements['weight_decay'].append(cfg.weight_decay)
    Experements['test_score'].append(test_score)
    Experements['val_score'].append(val_score)
    return Experements

In [386]:
model = GINModel(cfg).cuda()

optimizer = torch.optim.AdamW(model.parameters(),
                              lr = cfg.lr,
                              eps = cfg.eps,
                              betas = cfg.betas,
                              weight_decay = cfg.weight_decay)

criterion = nn.CrossEntropyLoss()

In [387]:
model

GINModel(
  (convs): ModuleList(
    (0): GINConv(nn=Linear(in_features=3703, out_features=80, bias=True))
  )
  (act): GELU(approximate='none')
  (drop): Dropout(p=0.0, inplace=False)
  (head): Linear(in_features=80, out_features=6, bias=True)
)

In [388]:
val_acc, test_acc = train(model, dataset, criterion, optimizer, cfg.max_epoches)
Experements = ExperementsUpgrade(cfg,test_acc,val_acc,Experements)

  0%|          | 0/80 [00:00<?, ?it/s]

0.138
0.644
0.692
0.758
0.772
0.774
0.78
0.784
Test Score 0.779


In [389]:
pd.DataFrame(Experements)

,epoches,lr,drop,depths,act,weight_decay,test_score,val_score
0,200,0.0003,0.0,"[3703, 64]",gelu,0.0002,0.775,0.766
1,150,0.0003,0.0,"[3703, 64]",gelu,0.0002,0.781,0.774
2,175,0.0003,0.0,"[3703, 64]",gelu,0.0002,0.777,0.772
3,125,0.0003,0.0,"[3703, 64]",gelu,0.0002,0.787,0.770
4,100,0.0003,0.0,"[3703, 64]",gelu,0.0002,0.792,0.774
5,75,0.0003,0.0,"[3703, 64]",gelu,0.0002,0.774,0.780
6,100,0.0003,0.0,"[3703, 128]",gelu,0.0002,0.790,0.774
7,75,0.0003,0.0,"[3703, 128]",gelu,0.0002,0.783,0.778
8,125,0.0003,0.0,"[3703, 128]",gelu,0.0002,0.778,0.774
9,100,0.0003,0.0,"[3703, 64, 32]",gelu,0.0002,0.775,0.772
